### Import Libraries

In [1]:
import rospy
import rospkg
from std_msgs.msg import Header
from trajectory_msgs.msg import JointTrajectory
from trajectory_msgs.msg import JointTrajectoryPoint
from sensor_msgs.msg import PointCloud2, Image
import sensor_msgs.point_cloud2 as pc2
import moveit_commander
import tf
from tf.transformations import *
import pcl
from pcl_helper import *
import sys
import numpy as np 
import os 
from geometry_msgs.msg import (
    PoseStamped,
    Pose,
    Point,
    Quaternion,
)
from gazebo_msgs.srv import (SpawnModel)

print("Packages Loaded")

Packages Loaded


### Functions for Clustering

In [2]:
# 복셀화(Down sampling)
def do_voxel_grid_downssampling(pcl_data,leaf_size):
    vox = pcl_data.make_voxel_grid_filter()
    vox.set_leaf_size(leaf_size, leaf_size, leaf_size)
    return  vox.filter()

# 노이즈 제거
def do_statistical_outlier_filtering(pcl_data,mean_k,tresh):
    outlier_filter = pcl_data.make_statistical_outlier_filter()
    outlier_filter.set_mean_k(mean_k)
    outlier_filter.set_std_dev_mul_thresh(tresh)
    return outlier_filter.filter()

# 바닥 제거 함수(random sample consensus 이용)
def do_ransac_plane_segmentation(pcl_data,pcl_sac_model_plane,pcl_sac_ransac,max_distance):
    '''
    Create the segmentation object
    :param pcl_data: point could data subscriber
    :param pcl_sac_model_plane: use to determine plane models, pcl.SACMODEL_PLANE
    :param pcl_sac_ransac: RANdom SAmple Consensus, pcl.SAC_RANSAC
    :param max_distance: Max distance for apoint to be considered fitting the model, 0.01
    :return: segmentation object
    '''
    seg = pcl_data.make_segmenter()
    seg.set_model_type(pcl_sac_model_plane)
    seg.set_method_type(pcl_sac_ransac)
    seg.set_distance_threshold(max_distance)

    # outliner 추출
    inliners, _ = seg.segment()
    inliner_object = pcl_data.extract(inliners,negative=False)
    outliner_object = pcl_data.extract(inliners,negative=True)
    return outliner_object

# clustering 함수
def euclid_cluster(cloud):
    white_cloud = XYZRGB_to_XYZ(cloud)
    tree = white_cloud.make_kdtree()
    ec = white_cloud.make_EuclideanClusterExtraction()
    ec.set_ClusterTolerance(0.015)
    ec.set_MinClusterSize(20)
    ec.set_MaxClusterSize(3000)
    ec.set_SearchMethod(tree)
    cluster_indices = ec.Extract()
    return cluster_indices, white_cloud

def cluster_mask(cluster_indices, white_cloud):
    cluster_color = get_color_list(len(cluster_indices))
    color_cluster_point_list = []
    for j, indices in enumerate(cluster_indices):
        for i, indice in enumerate(indices):
            color_cluster_point_list.append([
                                            white_cloud[indice][0],
                                            white_cloud[indice][1],
                                            white_cloud[indice][2],
                                            rgb_to_float( cluster_color[j] )
                                           ])
    cluster_cloud = pcl.PointCloud_PointXYZRGB()
    cluster_cloud.from_list(color_cluster_point_list)
    return cluster_cloud

# list를 pcd data로 바꾸는 함수
def change_list_to_pcd(lista):
    cloud = pcl.PointCloud_PointXYZRGB()
    cloud.from_list(lista)
    return cloud

# pcd data를 받아 중심점을 return하는 함수
def get_middle_point(pcd):
    x_total = 0
    y_total = 0
    z_total = []
    pcd_numpy = pcd.to_array()
    for i in range(len(pcd_numpy)):
        x_total += pcd_numpy[i,0]
        y_total += pcd_numpy[i,1]
        z_total.append(pcd_numpy[i,2])
    x = x_total / len(pcd_numpy)
    y = y_total / len(pcd_numpy)
    z = (max(z_total)+(min(z_total)))/2
    return x,y,z

# object 여러개를 리스트로 나누기
def get_obj_point(cluster_indices,white_cloud):
    obj_points = []
    for j,indices in enumerate(cluster_indices):
        point_list = []
        for k,indice in enumerate(indices):
            point_list.append([
                white_cloud[indice][0],
                white_cloud[indice][1],
                white_cloud[indice][2],
                1.5
            ])
        obj_points.append(point_list)
    return obj_points

# 관심 영역 설정
def do_passthrough(pcl_data,filter_axis,axis_min,axis_max):
    '''
    Create a PassThrough  object and assigns a filter axis and range.
    :param pcl_data: point could data subscriber
    :param filter_axis: filter axis
    :param axis_min: Minimum  axis to the passthrough filter object
    :param axis_max: Maximum axis to the passthrough filter object
    :return: passthrough on point cloud
    '''
    passthrough = pcl_data.make_passthrough_filter()
    passthrough.set_filter_field_name(filter_axis)
    passthrough.set_filter_limits(axis_min, axis_max)
    return passthrough.filter()

# 좌표 변환 함수
def tf_matrix():
    listener = tf.TransformListener() 
    listener.waitForTransform('world','camera_depth_optical_frame',rospy.Time(),rospy.Duration(2))
    (t,q) = listener.lookupTransform('world','camera_depth_optical_frame', rospy.Time(0))
    t_matrix = tf.transformations.translation_matrix(t)
    r_matrix = tf.transformations.quaternion_matrix(q)
    return np.dot(t_matrix,r_matrix)

def change_frame(matt, points):
    transpose = points[:,0:3]
    ones = np.ones((len(points),1))
    transpose = np.concatenate((transpose,ones),axis=1)
    transpose = transpose.T
    transpose_after = np.dot(matt,transpose)
    transpose_after = transpose_after.T
    transpose_after_after = transpose_after[:,0:3]
    rgb = points[:,3].reshape(len(points),1)
    finalmat = np.concatenate((transpose_after_after,rgb),axis=1)
    return finalmat

# numpy에서 pcd data로 바꾸는 함수
def numpy_to_pcd(nump):
    nump = nump.astype(np.float32)
    pcd = pcl.PointCloud_PointXYZI()
    pcd.from_array(nump)
    return pcd

# callback 함수
def callback(input_ros_msg):
    cloud = ros_to_pcl(input_ros_msg)
    cloud = do_voxel_grid_downssampling(cloud,0.005)
    delete_floor = do_ransac_plane_segmentation(cloud,pcl.SACMODEL_PLANE,pcl.SAC_RANSAC,0.01)
    delete_desk = do_ransac_plane_segmentation(delete_floor,pcl.SACMODEL_PLANE,pcl.SAC_RANSAC,0.01)

    delete_desk_1 = delete_desk.to_array()
    tf_mat = tf_matrix()
    delete_desk_points = change_frame(tf_mat,delete_desk_1)
    delete_desk_2 = numpy_to_pcd(delete_desk_points)
    delete_desk_2_filter = do_passthrough(delete_desk_2,'z',0.8,1.1)
    cluster_indices, white_cloud = euclid_cluster(delete_desk_2_filter)

    # cluster된 물체들 전부
    get_color_list.color_list = []
    final = cluster_mask(cluster_indices, white_cloud)
    final = pcl_to_ros(final)
    obj_points = get_obj_point(cluster_indices,white_cloud)
    middle_point_lists = []
    for i in range(len(obj_points)):
        obj_group_cloud = change_list_to_pcd(obj_points[i])
        x,y,z = get_middle_point(obj_group_cloud)
        middle_point = [x,y,z]
        middle_point_lists.append(middle_point)
        
    pub.publish(final)
    print("Number of Objects : {}".format(len(cluster_indices)))
    print("Middle point lists : ",middle_point_lists)

    return middle_point_lists

print("Done")

Done


### Functions for Moving

In [3]:
rospy.init_node('publish_joints')
arm_joint_names = ['shoulder_pan_joint','shoulder_lift_joint','elbow_joint', 
                'wrist_1_joint', 'wrist_2_joint','wrist_3_joint']
gripper_joint_names = ['gripper_finger1_joint']
rate    = rospy.Rate(1)
arm_pub = rospy.Publisher('/arm_controller/command', JointTrajectory, queue_size=10)
gripper_pub = rospy.Publisher('/gripper_controller/command', JointTrajectory, queue_size=10)

def move_gripper(joint):
    global gripper_joint_names, gripper_pub
    """ Open- Close Gripper """
    gripper        = JointTrajectory()
    gripper_value  = JointTrajectoryPoint()
    gripper.header = Header()
    gripper.joint_names = gripper_joint_names
    gripper_value.positions = [joint] # Open pose
    gripper_value.time_from_start = rospy.Duration.from_sec(0.01)
    gripper.points.append(gripper_value)
    gripper_pub.publish(gripper)


def move_arm(joint_seq): 
    Flag = True 
    global arm_joint_names, rate, arm_pub
    while Flag:
        """ Move to pick an object """
        for idx, joints in enumerate(joint_seq):
            joint = joints.reshape([6,])
            arm = JointTrajectory()
            arm_value = JointTrajectoryPoint()
            arm.header = Header()
            arm.joint_names= arm_joint_names
            arm_value.positions       = joint
            arm_value.time_from_start = rospy.Duration.from_sec(0.01)
            arm.points.append(arm_value)
            arm_pub.publish(arm)
            rate.sleep()    
            if idx+1 == len(joint_seq): 
                Flag = False 

In [4]:
""" Test """
joint_seq  = np.array([[2.0, -0.6596, 1.3364, 0.0350, 0, 0]]) 
move_arm(joint_seq)
move_gripper(0.9) # Open 

print("Publish trajectories")

Publish trajectories


### Functions for Spawning object

In [5]:
def spawn_gazebo_model(model_path, model_name, model_pose, reference_frame="world"):
  """
  Spawn model in gazebo
  """
  model_xml = ''
  with open(model_path, "r") as model_file:
    model_xml = model_file.read().replace('\n', '')
  rospy.wait_for_service('/gazebo/spawn_sdf_model')
  try:
    spawn_sdf = rospy.ServiceProxy('/gazebo/spawn_sdf_model', SpawnModel)
    resp_sdf= spawn_sdf(model_name, model_xml, "/", model_pose, reference_frame)
  except rospy.ServiceException:
    rospy.logerr("Spawn SDF service call failed")

moveit_commander.roscpp_initialize(sys.argv)
robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()

# Parameters 
table_height    = 0.79
# Cylinder 
cylinder_height = 0.2
cylinder_radius = 0.03
cylinder_pos_x  = 0.97 
cylinder_pos_y  = 0 
cylinder_pos_z  = table_height+cylinder_height/2.
# Box 
box_height      = 0.06
box_x           = 0.06 
box_y           = 0.06
box_pos_x       = 0.97 
box_pos_y       = 0 
box_pos_z       = table_height+box_height/2.

""" RViz """
# Cylinder 
cylinder_name = "cylinder"
cylinder_pos  = PoseStamped()
cylinder_pos.header.frame_id    = robot.get_planning_frame()
cylinder_pos.pose.position.x    = cylinder_pos_x
cylinder_pos.pose.position.y    = cylinder_pos_y
cylinder_pos.pose.position.z    = cylinder_pos_z
cylinder_pos.pose.orientation.w = 1.0  
# Box 
box_name = "box"
box_pos  = PoseStamped()
box_pos.header.frame_id    = robot.get_planning_frame()
box_pos.pose.position.x    = box_pos_x
box_pos.pose.position.y    = box_pos_y
box_pos.pose.position.z    = box_pos_z
box_pos.pose.orientation.w = 1.0  

""" GAZEBO """
rospack = rospkg.RosPack()
pack_path = rospack.get_path('ir_gazebo')
object_path = pack_path+os.sep+'urdf'+os.sep+'object'+os.sep+'cylinder.sdf'
object_name = cylinder_name
object_pose = Pose(position=Point(x=cylinder_pos_x , y=cylinder_pos_y, z=cylinder_pos_z))
object_name2 = 'obj2'
object_pose2 = Pose(position=Point(x=cylinder_pos_x+0.1 , y=cylinder_pos_y - 0.2, z=cylinder_pos_z))

[ INFO] [1653545557.474144932]: Loading robot model 'rilab_robot'...
[ INFO] [1653545557.475018028]: No root/virtual joint specified in SRDF. Assuming fixed joint


In [6]:
spawn_gazebo_model(object_path, object_name, object_pose)
spawn_gazebo_model(object_path, object_name2, object_pose2)

### Use Callback Function to get Middle Point Lists

In [7]:
import message_filters

class REALSENSE435():
    def __init__(self):    
        # Create member variables 
        self.objects = None 
        self.table = None 
        self.clusters = None 

        self.tick = 0
        self.point_cloud = None 
        self.image = None 

        self.clusters_publisher = rospy.Publisher("/pcl_cluster", PointCloud2, queue_size = 1)
        self.point_cloud_sub = message_filters.Subscriber('/camera/depth/color/points', PointCloud2)
        self.image_sub = message_filters.Subscriber('/camera/depth/image_raw', Image)
        self.ts = message_filters.TimeSynchronizer([self.point_cloud_sub, self.image_sub], 10)
        self.ts.registerCallback(self.callback)
        
        tic_temp = 0
        while self.tick<2:
            time.sleep(1e-3)
            tic_temp = tic_temp + 1
            if tic_temp > 5000:
                print ("[ERROR] CHECK REALSENSE435")
                break

    def callback(self, point_cloud_msg, image_msg):
        self.tick = self.tick+1
        self.depth_image = image_msg 
        self.point_cloud = point_cloud_msg 

    def cluster_publisher(self, clusters_msg):
        self.clusters_publisher.publish(clusters_msg)

In [8]:
realsense = REALSENSE435()
pub = rospy.Publisher("/camera/depth/color/points_new",PointCloud2,queue_size=1)
middle_point_lists = callback(realsense.point_cloud)

Number of Objects : 2
Middle point lists :  [[0.9522529944710667, 0.0006983283026910624, 0.8842407464981079], [1.052270383406908, -0.1963987345253671, 0.8838028907775879]]


In [ ]:
# If you want to check clustering in RViz
# rospy.Subscriber('/camera/depth/color/points', PointCloud2, callback)
# pub = rospy.Publisher("/camera/depth/color/points_new",PointCloud2,queue_size=1)
# rospy.spin()

### Move to the target object

In [9]:
# Get group_commander from MoveGroupCommander
group_name = robot.get_group_names()
move_group=moveit_commander.MoveGroupCommander(group_name[0])
move_gripper=moveit_commander.MoveGroupCommander(group_name[1])

[ INFO] [1653545572.335472495, 1123.045000000]: Ready to take commands for planning group arm.
[ INFO] [1653545572.809412384, 1123.505000000]: Ready to take commands for planning group gripper.


In [10]:
# Move using target_pose
pose_goal_1 = Pose()  
pose_goal_1.position.x = 0.6
pose_goal_1.position.y = 0.0
pose_goal_1.position.z = middle_point_lists[0][2] + 0.05

quat = quaternion_from_euler(0, math.pi, -math.pi/2)
pose_goal_1.orientation.x = quat[0]
pose_goal_1.orientation.y = quat[1]
pose_goal_1.orientation.z = quat[2]
pose_goal_1.orientation.w = quat[3]

move_group.set_pose_target(pose_goal_1)
plan = move_group.go(wait=True)

In [11]:
pose_goal_2 = Pose()  
pose_goal_2.position.x = middle_point_lists[0][0] - 0.28
pose_goal_2.position.y = middle_point_lists[0][1]
pose_goal_2.position.z = middle_point_lists[0][2] + 0.05

quat = quaternion_from_euler(0, -math.pi, -math.pi/2)
pose_goal_2.orientation.x = quat[0]
pose_goal_2.orientation.y = quat[1]
pose_goal_2.orientation.z = quat[2]
pose_goal_2.orientation.w = quat[3]

move_group.set_pose_target(pose_goal_2)
plan = move_group.go(wait=True)